# Feature Screening

### Imports

In [1]:
import pandas as pd
import numpy as np
import pickle
import os
import gc
import warnings
from collections import defaultdict
import matplotlib.pyplot as plt
import seaborn as sns

import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.stats.outliers_influence import variance_inflation_factor
from scipy.stats import pearsonr

warnings.filterwarnings("ignore")

### Configuration

In [ ]:
TARGET_VARIABLES = ['co2', 'gdp', 'primary_energy_consumption']
G20_COUNTRIES = [
    'United States', 'China', 'Japan', 'Germany', 
    'United Kingdom', 'France', 'Italy', 'Canada',
    'Brazil', 'Russia', 'India', 'Australia', 
    'Mexico', 'Indonesia', 'Turkey', 'Saudi Arabia',
    'South Africa', 'Argentina', 'South Korea'
]
# Top N number of features to consider
TOP_N = 40
# Minimum number of appearance in TOP N across metrics
MIN_APPEAR = 3
# To remove highly correlated features
CORR_THRESHOLD = 0.90
# Variance Inflation Factor threshold
VIF_THRESHOLD = 10.0

### Load data

In [3]:
# Load scores
def load_importance_scores(save_dir='data_export'):

    input_dir = os.path.join(save_dir, '02_results')
    screen_path = os.path.join(input_dir, 'importance_scores.pkl')

    try:
        with open(screen_path, 'rb') as f:
            importance_scores = pickle.load(f)
        print(f"loaded pickle")

        return importance_scores
    
    except FileNotFoundError:
        print(f"Run Stage 1 once again")
        return None

In [4]:
# Loading lagged data
def load_lag_data(save_dir='data_export'):

    lag_path = os.path.join(save_dir, 'lag_df_1965.pkl')

    try:
        lag_df = pd.read_pickle(lag_path)
        print(f"loaded lag data")
        return lag_df
    except FileNotFoundError:
        try:
            lag_csv = lag_path.replace('.pkl', '.csv')
            lag_df = pd.read_csv(lag_csv)
            return lag_df
        except FileNotFoundError as e:
            print(f"Error in loading {e}")
            return None

In [5]:
importance_scores = load_importance_scores()
importance_scores.keys()

loaded pickle


dict_keys(['co2', 'gdp', 'primary_energy_consumption'])

In [6]:
lag_df = load_lag_data()
lag_df.head(5)

loaded lag data


,country,year,primary_energy_consumption,population,energy_per_gdp,energy_per_capita,gdp,temperature_change_from_ghg,land_use_change_co2_per_capita,other_co2_per_capita,...,hydro_cons_change_pct_lag3,hydro_cons_change_pct_lag4,solar_share_energy_lag1,solar_share_energy_lag2,solar_share_energy_lag3,solar_share_energy_lag4,hydro_consumption_lag1,hydro_consumption_lag2,hydro_consumption_lag3,hydro_consumption_lag4
2567,Argentina,1965,328.528,22112635.0,1.452,14896.737,2.262849e+11,0.009,5.968,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2568,Argentina,1966,339.175,22453898.0,1.489,15139.622,2.278345e+11,0.009,5.804,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,3.623,NaN,NaN,NaN
2569,Argentina,1967,349.032,22799062.0,1.492,15337.348,2.339291e+11,0.009,8.292,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,3.669,3.623,NaN,NaN
2570,Argentina,1968,360.844,23150579.0,1.480,15612.201,2.438865e+11,0.010,5.502,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,3.757,3.669,3.623,NaN
2571,Argentina,1969,372.911,23508715.0,1.409,15887.447,2.647215e+11,0.010,4.045,NaN,...,1.291,NaN,NaN,NaN,NaN,NaN,4.429,3.757,3.669,3.623


### Utility Functions

In [7]:
def clear_memory():
    gc.collect()

## Removing highly correlated features

### Find features in Top N rank for all targets
wonder if i should include, there are some domain relevant features such like population are removed

In [ ]:
"""# Choosing top n-number of features by metric
def get_top_features(importance_scores, top_n=20):

    feature_appearance = {}

    for target in TARGET_VARIABLES:
        if target not in importance_scores:
            continue

        features = importance_scores[target]
        metrics = ['avg_abs_correlation', 'granger_significance_rate', 'avg_mutual_info']
        
        for metric in metrics:
            # Sort and rank
            sorted_features = sorted(features.items(), key=lambda x: x[1][metric])
            for rank, (feature, _) in enumerate(sorted_features[:top_n], 1):

                if feature not in feature_appearance:
                    feature_appearance[feature] = {
                        'ranks': [],
                        'targets': set()
                        }
                    
                feature_appearance[feature]['ranks'].append(rank)
                feature_appearance[feature]['targets'].add(target)

    feature_scores = {}

    for feature, data in feature_appearance.items():
        
        avg_rank = np.mean(data['ranks'])
        n_targets = len(data['targets'])
        
        feature_scores[feature] = {
            'avg_rank': avg_rank,
            'n_targets': n_targets
        }
    
    # Sort by combined score
    sorted_features = sorted(feature_scores.items(), key=lambda x: x[1]['n_targets'], reverse=True)
    
    print(f"\nTop features across all targets:")
    for i, (feat, scores) in enumerate(sorted_features[:], 1):
        print(f"{i:2d}. {feat:35} Targets: {scores['n_targets']}/3,   Avg Rank: {scores['avg_rank']:3.1f}")
    
    return sorted_features, feature_scores"""

In [ ]:
"""sorted_features, feature_scores = get_top_features(importance_scores, 20)"""


Top features across all targets:
 1. hydro_cons_change_twh               Targets: 3/3,   Avg Rank: 4.7
 2. renewables_cons_change_pct          Targets: 3/3,   Avg Rank: 3.7
 3. co2_including_luc_growth_abs        Targets: 3/3,   Avg Rank: 4.4
 4. coal_cons_change_pct                Targets: 3/3,   Avg Rank: 4.7
 5. nuclear_cons_change_twh             Targets: 3/3,   Avg Rank: 9.0
 6. coal_cons_change_twh                Targets: 3/3,   Avg Rank: 10.4
 7. co2_including_luc_growth_prct       Targets: 3/3,   Avg Rank: 5.9
 8. oil_prod_change_twh                 Targets: 3/3,   Avg Rank: 12.5
 9. coal_prod_change_twh                Targets: 3/3,   Avg Rank: 12.4
10. low_carbon_cons_change_pct          Targets: 3/3,   Avg Rank: 5.9
11. gas_cons_change_twh                 Targets: 3/3,   Avg Rank: 10.5
12. co2_growth_abs                      Targets: 3/3,   Avg Rank: 14.1
13. low_carbon_cons_change_twh          Targets: 3/3,   Avg Rank: 14.2
14. oil_cons_change_twh                 Targets: 3

In [ ]:
"""scores_appear_all = sorted_features[:24]"""

In [ ]:
"""features_appear_all = []
for i, (feature, _) in enumerate(scores_appear_all):
    features_appear_all.append(feature)"""

### Feature appearance
to check how many targets for each feature appear

In [37]:
def get_feature_app(importance_scores):
    
    feature_counts = {}
    for target in TARGET_VARIABLES:
        if target in importance_scores:
            for feature in importance_scores[target]:
                if feature not in feature_counts:
                    feature_counts[feature] = {'n_targets': 0, 'targets': []}
                feature_counts[feature]['n_targets'] += 1
                feature_counts[feature]['targets'].append(target)

    return feature_counts

### Domain relevant features
Features theoretically important to be kept

In [73]:
def domain_rel_features():

    domain = {
        'co2': [
            # direct emission
            'coal_consumption', 'oil_consumption', 'gas_consumption', 'fossil_fuel_consumption',
            # reduction
            'renewables_consumption', 'nuclear_consumption', 'low_carbon_consumption',
            # others
            'population', 'gdp', 'primary_energy_consumption', 'energy_per_capita', 'co2_per_unit_energy'
        ],
        'gdp': [
            'population',
            # energy - economy
            'primary_energy_consumption', 'electricity_generation', 'energy_per_gdp',
            # co2 - economy
            'trade_co2', 'consumption_co2', 'fossil_fuel_consumption', 'renewables_consumption'
        ],
        'primary_energy_consumption': [
            # sources
            'coal_consumption', 'oil_consumption', 'gas_consumption', 'renewables_consumption', 'nuclear_consumption', 'hydro_consumption',
            # economy
            'population', 'gdp',
            # intensity
            'energy_per_capita', 'energy_per_gdp', 'electricity_generation'
        ]
    }

    # Union all features
    union = set()
    for dom in domain.values():
        union.update(dom)

    return domain, list(union)

In [ ]:
# All features from importance scores
all_features = set()
for target in TARGET_VARIABLES:
    if target in importance_scores:
        all_features.update(importance_scores[target].keys())
        
all_features = list(all_features)
feature_counts = get_feature_app(importance_scores)

In [74]:
domain, all_relevant_features = domain_rel_features()

### Correlations Feature - Feature
Finding feature - feature correlations to remove highly correlated features

In [26]:
def ff_correlation(lag_df, countries, features):

    all_data = []

    for country in countries:
        country_data = lag_df[lag_df['country'] == country][features]
        all_data.append(country_data)

    combined_data = pd.concat(all_data, ignore_index=True)
    corr_matrix = combined_data.corr()

    return corr_matrix

In [27]:
def find_collinear_groups(lag_df, features, corr_threshold):

    correlation_matrix = ff_correlation(lag_df, G20_COUNTRIES, features)
    
    groups = []
    visited = set()
    
    for i, feat1 in enumerate(features):
        if feat1 in visited:
            continue
            
        group = [feat1]
        visited.add(feat1)
        
        for j, feat2 in enumerate(features):
            if feat2 not in visited and abs(correlation_matrix.loc[feat1, feat2]) > corr_threshold:
                group.append(feat2)
                visited.add(feat2)
        
        if len(group) > 1:
            groups.append(group)
    
    print("\nHighly correlated feature groups:")
    for i, group in enumerate(groups, 1):
        print(f"\nGroup {i}:")
        for feat in group:
            print(f"  - {feat}")
    
    return groups

In [78]:
find_collinear_groups(lag_df, all_features, CORR_THRESHOLD)


Highly correlated feature groups:

Group 1:
  - oil_energy_per_capita
  - fossil_energy_per_capita
  - oil_co2_per_capita

Group 2:
  - cumulative_coal_co2
  - share_global_cumulative_coal_co2
  - cumulative_co2

Group 3:
  - co2_per_capita
  - ghg_excluding_lucf_per_capita
  - fossil_elec_per_capita

Group 4:
  - coal_share_energy
  - coal_share_elec

Group 5:
  - ghg_per_capita
  - co2_including_luc_per_capita

Group 6:
  - wind_electricity
  - solar_electricity
  - solar_consumption
  - solar_cons_change_twh
  - wind_consumption

Group 7:
  - flaring_co2
  - share_global_flaring_co2

Group 8:
  - hydro_energy_per_capita
  - renewables_elec_per_capita
  - hydro_elec_per_capita
  - renewables_energy_per_capita

Group 9:
  - wind_share_energy
  - wind_share_elec

Group 10:
  - cement_co2
  - cumulative_cement_co2
  - coal_co2
  - coal_production
  - share_global_cement_co2
  - coal_consumption

Group 11:
  - renewables_electricity
  - renewables_consumption
  - hydro_electricity
  - l

[['oil_energy_per_capita', 'fossil_energy_per_capita', 'oil_co2_per_capita'],
 ['cumulative_coal_co2', 'share_global_cumulative_coal_co2', 'cumulative_co2'],
 ['co2_per_capita', 'ghg_excluding_lucf_per_capita', 'fossil_elec_per_capita'],
 ['coal_share_energy', 'coal_share_elec'],
 ['ghg_per_capita', 'co2_including_luc_per_capita'],
 ['wind_electricity',
  'solar_electricity',
  'solar_consumption',
  'solar_cons_change_twh',
  'wind_consumption'],
 ['flaring_co2', 'share_global_flaring_co2'],
 ['hydro_energy_per_capita',
  'renewables_elec_per_capita',
  'hydro_elec_per_capita',
  'renewables_energy_per_capita'],
 ['wind_share_energy', 'wind_share_elec'],
 ['cement_co2',
  'cumulative_cement_co2',
  'coal_co2',
  'coal_production',
  'share_global_cement_co2',
  'coal_consumption'],
 ['renewables_electricity',
  'renewables_consumption',
  'hydro_electricity',
  'low_carbon_electricity',
  'hydro_consumption'],
 ['other_renewables_energy_per_capita', 'other_renewables_elec_per_capita']

In [91]:
corr_matrix = ff_correlation(lag_df, G20_COUNTRIES, all_features)

In [90]:
def remove_high_corr(features, corr_matrix, feature_counts, features_to_keep, threshold):

    features_to_remove = set()
    corr_abs = corr_matrix.abs()

    # Finding highly correlated pairs
    for i in range(len(corr_abs.columns)):
        if corr_abs.columns[i] in features_to_remove:
            continue
        for j in range(i+1, len(corr_abs.columns)):
            if corr_abs.columns[j] in features_to_remove:
                continue

            if corr_abs.iloc[i, j] > threshold:
                
                feat1 = corr_abs.columns[i]
                feat2 = corr_abs.columns[j]

                # Keep the features
                if feat1 in features_to_keep and feat2 in features_to_keep:
                    continue

                if feat1 in features_to_keep:
                    features_to_remove.add(feat2)
                elif feat2 in features_to_keep:
                    features_to_remove.add(feat1)
                else:
                    # Keeping the one appears more in the targets
                    n_target1 = feature_counts.get(feat1, {}).get('n_targets', 0)
                    n_target2 = feature_counts.get(feat2, {}).get('n_targets', 0)

                    if n_target1 >= n_target2:
                        features_to_remove.add(feat2)
                    else:
                        features_to_remove.add(feat1)
    
    selected_features = [f for f in features if f not in features_to_remove]

    return selected_features, features_to_remove

In [94]:
selected_features, removed = remove_high_corr(all_features, corr_matrix, feature_counts, all_relevant_features, CORR_THRESHOLD)
selected_features

['oil_energy_per_capita',
 'electricity_share_energy',
 'cumulative_coal_co2',
 'co2_per_capita',
 'coal_share_energy',
 'ghg_per_capita',
 'wind_electricity',
 'oil_prod_per_capita',
 'flaring_co2',
 'hydro_energy_per_capita',
 'gas_prod_change_twh',
 'wind_share_energy',
 'other_renewables_energy_per_capita',
 'nuclear_share_energy',
 'co2_including_luc_per_unit_energy',
 'fossil_cons_change_pct',
 'other_renewable_consumption',
 'co2_growth_prct',
 'share_global_gas_co2',
 'co2_per_gdp',
 'low_carbon_energy_per_capita',
 'share_global_luc_co2',
 'renewables_consumption',
 'gas_prod_change_pct',
 'share_global_cumulative_luc_co2',
 'fossil_cons_change_twh',
 'oil_consumption',
 'coal_prod_per_capita',
 'other_renewables_share_energy',
 'nuclear_cons_change_twh',
 'renewables_cons_change_twh',
 'hydro_share_elec',
 'gas_energy_per_capita',
 'share_global_cumulative_cement_co2',
 'methane_per_capita',
 'gas_elec_per_capita',
 'gas_prod_per_capita',
 'cumulative_flaring_co2',
 'nitrous_

## Statistical Significance Testing

### Regression analysis
Testing each feature's effectiveness for each target with OLS regression

and remove features which are not signicant in enough countries or below 0.5 R-squared

In [95]:
def features_reg(lag_df, selected_features, p_threshold=0.05):

    reg_results = defaultdict(lambda: defaultdict(dict))

    for feature in selected_features:
        for target in TARGET_VARIABLES:

            sig_countries = 0
            r_squared_values = []
            p_values = []

            for country in G20_COUNTRIES:
                country_data = lag_df[lag_df['country'] == country]

                data_subset = country_data[[feature, target]].dropna()

                X = data_subset[feature].values
                y = data_subset[target].values

                Xc = sm.add_constant(X)

                model = sm.OLS(y, Xc).fit()

                p_value = model.pvalues[1]
                r_squared = model.rsquared

                if p_value < p_threshold:
                    sig_countries += 1

                r_squared_values.append(r_squared)
                p_values.append(p_value)


            if len(p_values) > 0:
                reg_results[feature][target] = {
                    'n_sig_countries': sig_countries,
                    'avg_r_squared': np.mean(r_squared_values),
                    'avg_p_value': np.mean(p_values)
                }

    features_to_remove = set()
    
    for feature in selected_features:
        keep_feature = False
        
        for target in TARGET_VARIABLES:
            if target in reg_results[feature]:
                results = reg_results[feature][target]
                
                # Keep if significant >= 10 countries
                if results['n_sig_countries'] >= 10:
                    keep_feature = True
                    break
                
                # Or if R-squared > 0.5
                if results['avg_r_squared'] >= 0.5:
                    keep_feature = True
                    break

        
        if not keep_feature:
            features_to_remove.add(feature)
            
            print(f"\n Removing {feature}:")
            for target in TARGET_VARIABLES:
                if target in reg_results[feature]:
                    res = reg_results[feature][target]
                    print(f"{target:27}: significant {res['n_sig_countries']}/{len(G20_COUNTRIES)} countries, "
                          f"avg R-sqaured = {res['avg_r_squared']:.3f}, avg p-value = {res['avg_p_value']}")
                    
    valid_features = [f for f in selected_features if f not in features_to_remove]

    summary_df = []
    for feature in valid_features:
        for target in TARGET_VARIABLES:
            if target in reg_results[feature]:
                res = reg_results[feature][target]
                summary_df.append({
                    'feature': feature,
                    'target': target,
                    'countries_significant': res['n_sig_countries'],
                    'avg_r_squared': res['avg_r_squared'],
                    'avg_p_value': res['avg_p_value']
                })
    
    summary_df = pd.DataFrame(summary_df)

    del reg_results, model, features_to_remove
    clear_memory()

    return valid_features, summary_df

In [119]:
valid_features_, reg_summary_df = features_reg(lag_df, selected_features, p_threshold=0.05)


 Removing gas_prod_change_pct:
co2                        : significant 9/19 countries, avg R-sqaured = 0.073, avg p-value = 0.17421660238860998
gdp                        : significant 9/19 countries, avg R-sqaured = 0.102, avg p-value = 0.18905244840930555
primary_energy_consumption : significant 8/19 countries, avg R-sqaured = 0.080, avg p-value = 0.17697722035617824

 Removing nuclear_cons_change_twh:
co2                        : significant 2/19 countries, avg R-sqaured = 0.063, avg p-value = nan
gdp                        : significant 5/19 countries, avg R-sqaured = 0.081, avg p-value = nan
primary_energy_consumption : significant 3/19 countries, avg R-sqaured = 0.051, avg p-value = nan

 Removing oil_prod_change_twh:
co2                        : significant 7/19 countries, avg R-sqaured = 0.061, avg p-value = nan
gdp                        : significant 9/19 countries, avg R-sqaured = 0.067, avg p-value = nan
primary_energy_consumption : significant 7/19 countries, avg R-sqaur

In [120]:
# Add all_relevant_features if they were removed
valid_features = valid_features_.copy()
for feat in all_relevant_features:
    if feat in selected_features and feat not in valid_features:

        # If it has importance
        importance = False

        for target in TARGET_VARIABLES:
            if target in importance_scores and feat in importance_scores[target]:
                
                scores = importance_scores[target][feat]
                if (scores.get('avg_abs_correlation', 0) > 0.1 or
                    scores.get('granger_significance_rate', 0) > 0.1):

                    importance = True
        
        if importance:
            valid_features.append(feat)
            print(f"{feat} is added back")

### Variance Inflation Factor
Use VIF to check multicollinearity

In [121]:
"""
VIF_i = 1 / (1 - R^2_i)
Generally, if vif >= 5 or 10, this might indicate there is a strong multicollinearity, 1 indicates no multicollinearity
"""
def check_vif(lag_df, features):
    
    all_data = []
    for country in G20_COUNTRIES:
        country_data = lag_df[lag_df['country'] == country][features].dropna()
        all_data.append(country_data)

    combined_data = pd.concat(all_data, ignore_index=True)

    # Calculating VIF
    vif_data = pd.DataFrame()
    vif_data["Feature"] = features
    vif_values = []

    for i in range(len(features)):
        try:
            vif = variance_inflation_factor(combined_data.values, i)
            vif_values.append(vif)

        except:
            vif_values.append(np.inf)

    vif_data["VIF"] = vif_values

    del all_data, combined_data
    clear_memory()

    return vif_data

In [122]:
vif_data = check_vif(lag_df, valid_features)

In [123]:
high_vif_features = vif_data[vif_data['VIF'] > VIF_THRESHOLD]['Feature'].tolist()
high_vif_features

['oil_energy_per_capita',
 'electricity_share_energy',
 'cumulative_coal_co2',
 'co2_per_capita',
 'coal_share_energy',
 'ghg_per_capita',
 'wind_electricity',
 'oil_prod_per_capita',
 'flaring_co2',
 'hydro_energy_per_capita',
 'wind_share_energy',
 'other_renewables_energy_per_capita',
 'nuclear_share_energy',
 'co2_including_luc_per_unit_energy',
 'other_renewable_consumption',
 'share_global_gas_co2',
 'co2_per_gdp',
 'low_carbon_energy_per_capita',
 'share_global_luc_co2',
 'renewables_consumption',
 'share_global_cumulative_luc_co2',
 'oil_consumption',
 'coal_prod_per_capita',
 'other_renewables_share_energy',
 'hydro_share_elec',
 'gas_energy_per_capita',
 'share_global_cumulative_cement_co2',
 'methane_per_capita',
 'gas_elec_per_capita',
 'gas_prod_per_capita',
 'cumulative_flaring_co2',
 'nitrous_oxide_per_capita',
 'oil_elec_per_capita',
 'low_carbon_share_energy',
 'fossil_fuel_consumption',
 'temperature_change_from_ch4',
 'wind_energy_per_capita',
 'oil_share_energy',
 '

In [124]:
features_to_remove = []
for feat in high_vif_features:

    vif_value = vif_data[vif_data['Feature'] == feat]['VIF'].values[0]
    print(f"{feat}: {vif_value:.2f}")

    if feat not in all_relevant_features:
        features_to_remove.append(feat)

    else:
        print(f"\nKeeping {feat}\n")

valid_features = [f for f in valid_features if f not in features_to_remove]

oil_energy_per_capita: 968.34
electricity_share_energy: 105.24
cumulative_coal_co2: 132.00
co2_per_capita: 939.83
coal_share_energy: 590.49
ghg_per_capita: 996.22
wind_electricity: 596.65
oil_prod_per_capita: 112.55
flaring_co2: 33.83
hydro_energy_per_capita: 763.24
wind_share_energy: 35.44
other_renewables_energy_per_capita: 46.34
nuclear_share_energy: 533.95
co2_including_luc_per_unit_energy: 88.85
other_renewable_consumption: 181.60
share_global_gas_co2: 442.89
co2_per_gdp: 435.71
low_carbon_energy_per_capita: 1803.44
share_global_luc_co2: 26.81
renewables_consumption: 4048179440333.03

Keeping renewables_consumption

share_global_cumulative_luc_co2: 122.66
oil_consumption: 16835886457459.80

Keeping oil_consumption

coal_prod_per_capita: 35.99
other_renewables_share_energy: 72.86
hydro_share_elec: 506.04
gas_energy_per_capita: 808.53
share_global_cumulative_cement_co2: 108.46
methane_per_capita: 163.59
gas_elec_per_capita: 96.63
gas_prod_per_capita: 50.04
cumulative_flaring_co2: 33

In [125]:
valid_features

['gas_prod_change_twh',
 'fossil_cons_change_pct',
 'co2_growth_prct',
 'renewables_consumption',
 'fossil_cons_change_twh',
 'oil_consumption',
 'renewables_cons_change_twh',
 'fossil_fuel_consumption',
 'gas_cons_change_pct',
 'wind_cons_change_twh',
 'energy_per_capita',
 'oil_cons_change_pct',
 'co2_per_unit_energy',
 'low_carbon_consumption',
 'electricity_generation',
 'solar_share_energy',
 'population',
 'other_renewables_cons_change_twh',
 'coal_consumption',
 'hydro_consumption',
 'energy_per_gdp',
 'nuclear_consumption',
 'gas_consumption']

## Analysis

In [134]:
def feature_summary(valid_features, importance_scores, reg_summary_df):

    summary = []

    for feature in valid_features:
        feature_info = {
            'feature': feature,
            'appears_in_targets': 0,
            'targets': [],
            'avg_correlation': 0,
            'avg_granger_sig': 0,
            'avg_mi': 0,
            'regression_sig': 0
        }

        corr_values = []
        granger_values = []
        mi_values = []

        for target in TARGET_VARIABLES:
            if target in importance_scores and feature in importance_scores[target]:
                feature_info['appears_in_targets'] += 1
                feature_info['targets'].append(target)

                scores = importance_scores[target][feature]
                corr_values.append(scores.get('avg_abs_correlation', 0))
                granger_values.append(scores.get('granger_significance_rate', 0))
                mi_values.append(scores.get('avg_mutual_info', 0))

        feature_info['avg_correlation'] = np.mean(corr_values) if corr_values else 0
        feature_info['avg_granger_sig'] = np.mean(granger_values) if granger_values else 0
        feature_info['avg_mi'] = np.mean(mi_values) if mi_values else 0

        reg_count = len(reg_summary_df[reg_summary_df['feature'] == feature])
        feature_info['regression_sig'] = reg_count

        summary.append(feature_info)

    summary_df = pd.DataFrame(summary)
    summary_df = summary_df.sort_values(['appears_in_targets', 'avg_correlation'], ascending=[False, False])

    return summary_df

In [135]:
summary_df = feature_summary(valid_features, importance_scores, reg_summary_df)

In [136]:
summary_df

,feature,appears_in_targets,targets,avg_correlation,avg_granger_sig,avg_mi,regression_sig
7,fossil_fuel_consumption,3,"[co2, gdp, primary_energy_consumption]",0.920757,0.473684,1.818021,3
16,population,3,"[co2, gdp, primary_energy_consumption]",0.877415,0.614035,1.702223,3
14,electricity_generation,3,"[co2, gdp, primary_energy_consumption]",0.874056,0.421053,1.355025,3
22,gas_consumption,3,"[co2, gdp, primary_energy_consumption]",0.847214,0.456140,1.475571,3
10,energy_per_capita,3,"[co2, gdp, primary_energy_consumption]",0.846111,0.245614,1.555621,3
18,coal_consumption,3,"[co2, gdp, primary_energy_consumption]",0.832639,0.368421,1.370310,3
5,oil_consumption,3,"[co2, gdp, primary_energy_consumption]",0.824997,0.350877,1.410547,3
13,low_carbon_consumption,3,"[co2, gdp, primary_energy_consumption]",0.818156,0.508772,1.249805,3
3,renewables_consumption,3,"[co2, gdp, primary_energy_consumption]",0.732484,0.491228,1.061657,3
12,co2_per_unit_energy,3,"[co2, gdp, primary_energy_consumption]",0.661884,0.263158,0.854495,3


## Visualisation

In [176]:
def plot_results(valid_features, corr_matrix, summary_df, save_dir='data_export/02_plots'):

    output_dir = os.path.join(save_dir, '02_feature_screen')
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # Summary plot
    fig, axes = plt.subplots(2, 2, figsize=(20, 12))

    # Correlation
    ax = axes[0, 0]
    top20_corr = summary_df.nlargest(20, 'avg_correlation')
    ax.barh(range(len(top20_corr)), top20_corr['avg_correlation'])
    ax.set_yticks(range(len(top20_corr)))
    ax.set_yticklabels(top20_corr['feature'], fontsize=7)
    ax.set_xlabel('Average Correlation')
    ax.set_title('Top 20 Features by Correlation')
    ax.invert_yaxis()

    # Granger significance
    ax = axes[0, 1]
    top20_granger = summary_df.nlargest(20, 'avg_granger_sig')
    ax.barh(range(len(top20_granger)), top20_granger['avg_granger_sig'])
    ax.set_yticks(range(len(top20_granger)))
    ax.set_yticklabels(top20_granger['feature'], fontsize=7)
    ax.set_xlabel('Average Granger Significance Rate')
    ax.set_title('Top 20 Features by Granger Causality')
    ax.invert_yaxis()

    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, 'screening_summary.png'), bbox_inches='tight')
    plt.close()

    # Correlation heatmap sampled 30
    if len(valid_features) > 30:
        sample_features = summary_df.head(30)['feature'].tolist()
    else:
        sample_features = valid_features
    
    plt.figure(figsize=(24, 20))
    selected_corr = corr_matrix.loc[sample_features, sample_features]
    
    mask = np.triu(np.ones_like(selected_corr, dtype=bool))
    sns.heatmap(selected_corr, mask=mask, annot=True, cmap='coolwarm', 
                center=0, square=True, linewidths=0.5, annot_kws={"fontsize": 6})
    plt.xticks(rotation=30)
    plt.title('Feature Correlation Matrix After Screening')
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, 'screen_correlation_matrix.png'), bbox_inches='tight')
    plt.close()

In [177]:
plot_results(valid_features, corr_matrix, summary_df)

In [180]:
# Save valid features and summary df
def save_features_df(valid_features, summary_df, save_dir='data_export'):

    output_dir = os.path.join(save_dir, '02_results')
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    with open(os.path.join(output_dir, 'selected_features.pkl'), 'wb') as f:
        pickle.dump(valid_features, f)

    summary_df.to_csv(os.path.join(output_dir, 'screen_summary.csv'), index=False)

In [181]:
save_features_df(valid_features, summary_df)